<a href="https://colab.research.google.com/github/Worachet-Ch/Project/blob/main/XGboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/project2022' #เข้า drive
data_test = pd.read_csv(os.path.join(path,'test15.csv'))
data_test.drop('Unnamed: 0',1,inplace = True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-8-22381b44fb0a>:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_test.drop('Unnamed: 0',1,inplace = True)


In [ ]:
from xgboost import XGBRegressor
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score

In [ ]:
data_test

In [ ]:
def data(Train, Test):
  y_train =Train[['telomere length (kb) ']].values.copy() #n = 408
  X_train = Train.drop('telomere length (kb) ',axis = 1).values.copy() # n =408
  X_test = Test.drop('telomere length (kb) ',axis = 1).values.copy()
  y_test = Test[['telomere length (kb) ']].values.copy()
  model = XGBRegressor()
  param = {'learning_rate': [0.01, 0.1,0.05],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.3,0.5, 0.7,1],
        'colsample_bytree': [0.1, 0.3, 0.5, 0.7, 0.9],
        'n_estimators' : [100,200,300, 500,700,1000],
        'seed':[216],'alpha':[0.1, 0.3, 0.5, 0.7, 0.9]}
  search = GridSearchCV(model, param, scoring='neg_mean_absolute_error', n_jobs=1, cv=5, refit=True)
  result = search.fit(X_train, y_train)
  best_model = result.best_estimator_
  best_model.fit(X_train,y_train) 
  scores_mae = cross_val_score(best_model , X_train, y_train, scoring='neg_mean_absolute_error',cv=5 )
  y_pred = best_model.predict(X_test)
  mae  = metrics.mean_absolute_error(y_test, y_pred)
  print('Best model:',best_model)
  print(f'score:{scores_mae}')
  print(f'Mean score:{scores_mae.mean()}')
  print('MAE Test:',mae)

# พร้อมกันทุกตัวแปร

In [ ]:
df_once = pd.read_csv(os.path.join(path,'df_once.csv'))
df_once.drop('Unnamed: 0',1,inplace = True)

In [ ]:
df_once

In [ ]:
data(df_once,data_test)

# ทีละตัวแปร จากค่าว่างมากไปน้อย

In [ ]:
df_decreasing = pd.read_csv(os.path.join(path,'df_decreasing.csv'))
df_decreasing.drop('Unnamed: 0',1,inplace = True)

In [ ]:
df_decreasing

In [ ]:
data(df_decreasing,data_test)

# ทีละตัวแปร จากค่าว่างน้อยไปมาก

In [ ]:
df_increasing = pd.read_csv(os.path.join(path,'df_increasing.csv'))
df_increasing.drop('Unnamed: 0.1',1,inplace = True)
df_increasing.drop('Unnamed: 0',1,inplace = True)

In [ ]:
df_increasing

In [ ]:
data(df_increasing,data_test)

# ข้อมูลสุ่มเติมชุด 1

In [ ]:
df_Random_1 = pd.read_csv(os.path.join(path,'df_Random_1.csv'))
df_Random_1.drop('Unnamed: 0',1,inplace = True)

In [ ]:
df_Random_1

In [ ]:
data(df_Random_1,data_test)

# ข้อมูลสุ่มเติมชุด 2

In [ ]:
df_Random_2 = pd.read_csv(os.path.join(path,'df_Random_2.csv'))
df_Random_2.drop('Unnamed: 0',1,inplace = True)

In [ ]:
df_Random_2

In [ ]:
data(df_Random_2,data_test)

# ข้อมูลควบคุม

In [ ]:
df_control = pd.read_csv(os.path.join(path,'control.csv'))
df_control.drop('Unnamed: 0',1,inplace = True)

In [ ]:
df_control

In [ ]:
df_control